In [1]:
from pyDeepInsight import ImageTransformer, CAMFeatureSelector

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import torch
import torchvision.transforms as transforms
import timm
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from geomstats.learning.frechet_mean import FrechetMean
from geomstats.learning.pca import TangentPCA
from geomstats.geometry.hypersphere import Hypersphere
from IPython.display import display, clear_output
import torchvision.models as models
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import roc_auc_score, ConfusionMatrixDisplay, confusion_matrix
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, roc_auc_score, roc_curve
import warnings

warnings.filterwarnings("ignore")

INFO: Using numpy backend


In [2]:
data = pd.read_csv('ibd_data_set.csv',index_col=0)
data = data.div(data.sum(axis=1), axis=0) #Convert the data into compositional data through division by the total sum.
data = np.sqrt(data) #Apply a square-root transformation to the data to map it into directional space.
X=data.values 
sample=pd.read_csv('ibd_response.csv',index_col=0)
y=sample['ibd'] 
le = LabelEncoder()
y_enc = le.fit_transform(y)
num_classes = np.unique(y_enc).size

In [3]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score
from sklearn.utils import shuffle
import optuna

# Apply 10-Fold Cross-Validation
kf = KFold(n_splits=10, shuffle=True)

# Lists to store accuracy and AUC
accuracies = []
aucs = []

le = LabelEncoder()  # Label encoder

pixel_size = (224, 224)
sphere = Hypersphere(dim=36348)  # Declare a hypersphere with one dimension lower than the actual dimension for Tangent PCA
it = ImageTransformer(
    feature_extractor=TangentPCA(sphere, n_components=2),
    pixels=pixel_size) # The default center point for TangentPCA is the Frechet mean

# Create an Optuna study
study = optuna.create_study(direction='maximize')

# Define a trial function for Optuna to optimize lr and weight_decay
def optimize_trial(trial):
    # Set learning rate and weight_decay as hyperparameters to optimize using Optuna
    lr = trial.suggest_float('lr', 1e-6, 1e-2, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-2, log=True)
    
    actual_labels_list = []
    probs_list = []
    
    for train_index, test_index in kf.split(X):
        # Train/Test Split
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Apply ImageTransformer (with exception handling)
        try:
            it.fit(X_train, y=y_train, plot=False)
        except Exception as e:
            print(f"SVD 수렴 문제 발생: {e}. 계속 진행합니다.")
            continue  # 수렴 문제 발생 시 다음 루프로 넘어감

        X_train_img = it.transform(X_train)
        X_test_img = it.transform(X_test)

        fdm = it.feature_density_matrix()
        fdm_nonzero_indices = np.nonzero(fdm)
        X_train_img = it.transform(X_train)
        X_test_img = it.transform(X_test)

        preprocess = transforms.Compose([
            transforms.ToTensor()
        ])

        # Convert to Tensor
        X_train_tensor = torch.stack([preprocess(img) for img in X_train_img]).float()
        y_train_tensor = torch.from_numpy(le.fit_transform(y_train))

        X_test_tensor = torch.stack([preprocess(img) for img in X_test_img]).float()
        y_test_tensor = torch.from_numpy(le.transform(y_test))

        # Set up DataLoader
        batch_size = 32
        trainset = TensorDataset(X_train_tensor, y_train_tensor)
        trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)

        testset = TensorDataset(X_test_tensor, y_test_tensor)
        testloader = DataLoader(testset, batch_size=1, shuffle=False)

        model = models.resnet50(weights='DEFAULT')
        
        # Modify the last fully connected layer for binary classification
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, 1)  # Output layer for binary classification
        
        # Define loss function and optimizer
        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

        # Move model to GPU
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)

        # Start training
        epochs = 100 
        for epoch in range(epochs):
            model.train()
            running_loss = 0.0

            for i, data in enumerate(trainloader, 0):
                inputs, labels = data
                inputs = inputs.to(device)
                labels = labels.to(device).float()

                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels.unsqueeze(1))
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
        
        # 평가 모드로 전환
        model.eval()

        with torch.no_grad():
            for data in testloader:
                inputs, labels = data
                inputs = inputs.to(device)
                labels = labels.to(device).float()

                outputs = model(inputs)
                probs = torch.sigmoid(outputs).cpu().numpy()

                # Store actual labels and probabilities for AUC calculation
                actual_labels_list.append(labels.cpu().numpy()[0])
                probs_list.append(probs[0])

    # AUC 계산
    try:
        auc = roc_auc_score(actual_labels_list, probs_list)
    except ValueError:
        # Handle exceptions if AUC calculation fails
        print("AUC 계산에 실패했습니다.")
        auc = 0.0

    # Optuna aims to maximize the objective function, so return AUC
    return auc

# Run Optuna optimization
study.optimize(optimize_trial, n_trials=1000)

# Print optimization results
print(f'Best trial AUC: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

[I 2024-10-12 14:11:53,313] A new study created in memory with name: no-name-9f4efa64-564a-452d-b66c-3f81789a49b7


SVD 수렴 문제 발생: SVD did not converge. 계속 진행합니다.


[I 2024-10-12 14:16:09,559] Trial 0 finished with value: 0.6537878787878788 and parameters: {'lr': 0.00016633333694351636, 'weight_decay': 8.159105651079604e-05}. Best is trial 0 with value: 0.6537878787878788.


SVD 수렴 문제 발생: SVD did not converge. 계속 진행합니다.


[I 2024-10-12 14:20:39,579] Trial 1 finished with value: 0.6065573770491803 and parameters: {'lr': 0.0029138433131604673, 'weight_decay': 0.0010419721171673678}. Best is trial 0 with value: 0.6537878787878788.


...

SVD 수렴 문제 발생: SVD did not converge. 계속 진행합니다.


[I 2024-10-15 17:48:45,311] Trial 997 finished with value: 0.6604215456674474 and parameters: {'lr': 7.316534875579856e-06, 'weight_decay': 1.2282872523349148e-05}. Best is trial 490 with value: 0.8166666666666667.


SVD 수렴 문제 발생: SVD did not converge. 계속 진행합니다.


[I 2024-10-15 17:53:29,911] Trial 998 finished with value: 0.6548387096774193 and parameters: {'lr': 5.628002060489833e-06, 'weight_decay': 7.644121659463987e-06}. Best is trial 490 with value: 0.8166666666666667.
[I 2024-10-15 17:58:14,414] Trial 999 finished with value: 0.5628787878787879 and parameters: {'lr': 7.004414962479449e-06, 'weight_decay': 9.964016157469509e-06}. Best is trial 490 with value: 0.8166666666666667.


SVD 수렴 문제 발생: SVD did not converge. 계속 진행합니다.
Best trial AUC: 0.8166666666666667
Best hyperparameters: {'lr': 5.953085385304724e-06, 'weight_decay': 3.3129820853775895e-06}
